In [1]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import tensorflow as tf
import tensorflow_ranking as tfr
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

2025-02-25 19:40:18.491161: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-25 19:40:18.974084: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-25 19:40:18.974169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-25 19:40:19.060349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 19:40:19.245864: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-25 19:40:19.248788: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# ダウンロード先ディレクトリ
data_dir = "data"

# 各CSVファイルのパス
race_result_csv     = os.path.join(data_dir, "19860105-20210731_race_result.csv")
# odds_csv            = os.path.join(data_dir, "19860105-20210731_odds.csv")

# pandas DataFrameとして読み込み
df_race_result   = pd.read_csv(race_result_csv)
# df_odds          = pd.read_csv(odds_csv)

print(len(df_race_result))

/tmp/ipykernel_9851/1984880406.py:9: DtypeWarning: Columns (8,11,15,16,17,18,19,20,21,22,23,24,25,26,27,34,36,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_race_result   = pd.read_csv(race_result_csv)


1626811


In [3]:
df_race_result.head()

,レース馬番ID,レースID,レース日付,開催回数,競馬場コード,競馬場名,開催日数,競争条件,レース記号/[抽],レース記号/(馬齢),...,4コーナー,上り,単勝,人気,馬体重,場体重増減,東西・外国・地方区分,調教師,馬主,賞金(万円)
0,19860101010102,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),...,1.0,38.6,2.1,1.0,468.0,0.0,東,宮沢今朝,アイ・ケイ・テイ・オーナーズ,290.0
1,19860101010103,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),...,4.0,40.3,7.0,4.0,430.0,4.0,東,斎藤籌敬,松井健一,120.0
2,19860101010105,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),...,3.0,40.6,59.1,6.0,460.0,-4.0,西,境直行,塚原金治,73.0
3,19860101010106,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),...,2.0,41.2,2.1,2.0,456.0,14.0,西,坂口正大,鈴木隆,44.0
4,19860101010107,198601010101,1986-06-07,1,1,札幌,1,4歳以上300万下,[抽],(馬齢),...,4.0,41.7,6.2,3.0,432.0,-6.0,西,荻野光男,勢戸澄雄,29.0


In [4]:
# df_odds.head()

In [5]:
print(df_race_result.columns)

Index(['レース馬番ID', 'レースID', 'レース日付', '開催回数', '競馬場コード', '競馬場名', '開催日数', '競争条件',
       'レース記号/[抽]', 'レース記号/(馬齢)', 'レース記号/牝', 'レース記号/(父)', 'レース記号/(別定)',
       'レース記号/(混)', 'レース記号/(ハンデ)', 'レース記号/(抽)', 'レース記号/(市)', 'レース記号/(定量)',
       'レース記号/牡', 'レース記号/関東配布馬', 'レース記号/(指)', 'レース記号/関西配布馬', 'レース記号/九州産馬',
       'レース記号/見習騎手', 'レース記号/せん', 'レース記号/(国際)', 'レース記号/[指]', 'レース記号/(特指)',
       'レース番号', '重賞回次', 'レース名', 'リステッド・重賞競走', '障害区分', '芝・ダート区分', '芝・ダート区分2',
       '右左回り・直線区分', '内・外・襷区分', '距離(m)', '天候', '馬場状態1', '馬場状態2', '発走時刻', '着順',
       '着順注記', '枠番', '馬番', '馬名', '性別', '馬齢', '斤量', '騎手', 'タイム', '着差', '1コーナー',
       '2コーナー', '3コーナー', '4コーナー', '上り', '単勝', '人気', '馬体重', '場体重増減',
       '東西・外国・地方区分', '調教師', '馬主', '賞金(万円)'],
      dtype='object')


In [6]:
# print(df_odds.columns)

In [7]:
df_race_result[['障害区分', '芝・ダート区分', '芝・ダート区分2','右左回り・直線区分', '内・外・襷区分', '距離(m)', '天候', '馬場状態1', '馬場状態2']].head()

,障害区分,芝・ダート区分,芝・ダート区分2,右左回り・直線区分,内・外・襷区分,距離(m),天候,馬場状態1,馬場状態2
0,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN
1,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN
2,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN
3,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN
4,NaN,ダート,NaN,右,NaN,1500,晴,良,NaN


障害レースは考えないので障害レースの情報を落とす。

In [8]:
df_race_result.drop(df_race_result[df_race_result['障害区分'].notna()].index, inplace=True)
df_race_result.drop('障害区分', axis=1, inplace=True)
print(len(df_race_result))

1572791


メインのカラムに絞る。

In [9]:
main_columns = ['レースID', 'レース日付', '競馬場名',
                'リステッド・重賞競走', '右左回り・直線区分', '距離(m)', '馬場状態1', '着順',
                '着順注記', '馬番', '馬名', '馬齢', '斤量', '単勝', '人気', '馬体重']

df_race_result = df_race_result[main_columns]
df_race_result.head()

,レースID,レース日付,競馬場名,リステッド・重賞競走,右左回り・直線区分,距離(m),馬場状態1,着順,着順注記,馬番,馬名,馬齢,斤量,単勝,人気,馬体重
0,198601010101,1986-06-07,札幌,NaN,右,1500,良,1.0,NaN,2,ワクセイ,4,55.0,2.1,1.0,468.0
1,198601010101,1986-06-07,札幌,NaN,右,1500,良,2.0,NaN,3,マツタカラオー,4,55.0,7.0,4.0,430.0
2,198601010101,1986-06-07,札幌,NaN,右,1500,良,3.0,NaN,5,カンキョウヘルス,4,55.0,59.1,6.0,460.0
3,198601010101,1986-06-07,札幌,NaN,右,1500,良,4.0,NaN,6,スズタカエース,5,57.0,2.1,2.0,456.0
4,198601010101,1986-06-07,札幌,NaN,右,1500,良,5.0,NaN,7,クリヤーパーマン,4,55.0,6.2,3.0,432.0


着順注記がある特殊なレース（出走取消等があったレース）を除外する。

In [10]:
df_race_result.drop(df_race_result[df_race_result['着順注記'].notna()].index, inplace=True)
df_race_result.drop('着順注記', axis=1, inplace=True)
print(len(df_race_result))

1560664


重賞以外のレースについてNaNの処理を行う。

In [11]:
# df_race_result.drop(df_race_result[df_race_result['リステッド・重賞競走'].isna()].index, inplace=True)
df_race_result['リステッド・重賞競走'].fillna('グレード無し', inplace=True)
print(len(df_race_result))
df_race_result.head()

1560664


/tmp/ipykernel_9851/2707375020.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_race_result['リステッド・重賞競走'].fillna('グレード無し', inplace=True)


,レースID,レース日付,競馬場名,リステッド・重賞競走,右左回り・直線区分,距離(m),馬場状態1,着順,馬番,馬名,馬齢,斤量,単勝,人気,馬体重
0,198601010101,1986-06-07,札幌,グレード無し,右,1500,良,1.0,2,ワクセイ,4,55.0,2.1,1.0,468.0
1,198601010101,1986-06-07,札幌,グレード無し,右,1500,良,2.0,3,マツタカラオー,4,55.0,7.0,4.0,430.0
2,198601010101,1986-06-07,札幌,グレード無し,右,1500,良,3.0,5,カンキョウヘルス,4,55.0,59.1,6.0,460.0
3,198601010101,1986-06-07,札幌,グレード無し,右,1500,良,4.0,6,スズタカエース,5,57.0,2.1,2.0,456.0
4,198601010101,1986-06-07,札幌,グレード無し,右,1500,良,5.0,7,クリヤーパーマン,4,55.0,6.2,3.0,432.0


カテゴリ変数のone-hotエンコーディングの実行

In [12]:
categorical_columns = ['競馬場名', 'リステッド・重賞競走', '右左回り・直線区分', '馬場状態1', '馬番', '人気']
df_encoded = pd.get_dummies(df_race_result, columns=categorical_columns)
bool_cols = df_encoded.select_dtypes(include='bool').columns
df_encoded[bool_cols] = df_encoded[bool_cols].astype(int)
df_race_result = df_encoded
df_race_result.head()

,レースID,レース日付,距離(m),着順,馬名,馬齢,斤量,単勝,馬体重,競馬場名_中京,...,人気_15.0,人気_16.0,人気_17.0,人気_18.0,人気_19.0,人気_20.0,人気_21.0,人気_22.0,人気_23.0,人気_24.0
0,198601010101,1986-06-07,1500,1.0,ワクセイ,4,55.0,2.1,468.0,0,...,0,0,0,0,0,0,0,0,0,0
1,198601010101,1986-06-07,1500,2.0,マツタカラオー,4,55.0,7.0,430.0,0,...,0,0,0,0,0,0,0,0,0,0
2,198601010101,1986-06-07,1500,3.0,カンキョウヘルス,4,55.0,59.1,460.0,0,...,0,0,0,0,0,0,0,0,0,0
3,198601010101,1986-06-07,1500,4.0,スズタカエース,5,57.0,2.1,456.0,0,...,0,0,0,0,0,0,0,0,0,0
4,198601010101,1986-06-07,1500,5.0,クリヤーパーマン,4,55.0,6.2,432.0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
print(df_race_result.columns)

Index(['レースID', 'レース日付', '距離(m)', '着順', '馬名', '馬齢', '斤量', '単勝', '馬体重',
       '競馬場名_中京', '競馬場名_中山', '競馬場名_京都', '競馬場名_函館', '競馬場名_小倉', '競馬場名_新潟',
       '競馬場名_札幌', '競馬場名_東京', '競馬場名_福島', '競馬場名_阪神', 'リステッド・重賞競走_G',
       'リステッド・重賞競走_G1', 'リステッド・重賞競走_G2', 'リステッド・重賞競走_G3', 'リステッド・重賞競走_L',
       'リステッド・重賞競走_グレード無し', '右左回り・直線区分_右', '右左回り・直線区分_左', '右左回り・直線区分_直線',
       '馬場状態1_不良', '馬場状態1_稍重', '馬場状態1_良', '馬場状態1_重', '馬番_1', '馬番_2', '馬番_3',
       '馬番_4', '馬番_5', '馬番_6', '馬番_7', '馬番_8', '馬番_9', '馬番_10', '馬番_11',
       '馬番_12', '馬番_13', '馬番_14', '馬番_15', '馬番_16', '馬番_17', '馬番_18', '馬番_19',
       '馬番_20', '馬番_21', '馬番_22', '馬番_23', '馬番_24', '人気_1.0', '人気_2.0',
       '人気_3.0', '人気_4.0', '人気_5.0', '人気_6.0', '人気_7.0', '人気_8.0', '人気_9.0',
       '人気_10.0', '人気_11.0', '人気_12.0', '人気_13.0', '人気_14.0', '人気_15.0',
       '人気_16.0', '人気_17.0', '人気_18.0', '人気_19.0', '人気_20.0', '人気_21.0',
       '人気_22.0', '人気_23.0', '人気_24.0'],
      dtype='object')


スケーリングを行う（手動）。

ロバストスケーラーやミニマックススケーラーで行うとNaNが出現してしまったので、numpyのlog1pと手動のスケーリングを以下で行う。\
ログ付与後の正値を$x$、スケーリング後の正値を$x_{scaled}$とすると、以下の式を適用する。なお、$x_{max}, x_{min}$はそれぞれの属性の最大値、最小値である。


$x_{scaled}=\frac{x-x_{min}}{x_{max}-x_{min}}$

In [14]:
# scaler = MinMaxScaler(feature_range=(0, 1))
real_valued_columns = ['距離(m)', '馬齢', '斤量', '単勝', '馬体重']
real_valued_data = np.log1p(df_race_result[real_valued_columns])
# scaled_real_valued_data = scaler.fit_transform(real_valued_data)
scaled_real_valued_df = pd.DataFrame(real_valued_data, columns=real_valued_columns)
df_race_result[real_valued_columns] = scaled_real_valued_df[real_valued_columns]
min_vals = df_race_result[real_valued_columns].min()
max_vals = df_race_result[real_valued_columns].max()
diff_vals = max_vals - min_vals
scaled_data = (df_race_result[real_valued_columns] - min_vals) / diff_vals
df_race_result[real_valued_columns] = scaled_data
num_nan = df_race_result[real_valued_columns].isnull().sum()
distance = df_race_result['距離(m)'].unique()
print(num_nan)
print(distance)
# print(min_vals)
# print(max_vals)
# print(diff_vals)
df_race_result.head()

距離(m)     0
馬齢        0
斤量        0
単勝        0
馬体重      44
dtype: int64
[0.31645698 0.         0.14228505 0.45878536 0.54104126 0.41416326
 0.7152652  0.68339131 0.74588929 0.36683682 0.2626024  0.61545451
 0.0743778  0.57913367 0.65016127 0.90802436 1.         0.85762826
 0.50099555 0.8037551  0.20475803 0.10906914 0.95536515]


,レースID,レース日付,距離(m),着順,馬名,馬齢,斤量,単勝,馬体重,競馬場名_中京,...,人気_15.0,人気_16.0,人気_17.0,人気_18.0,人気_19.0,人気_20.0,人気_21.0,人気_22.0,人気_23.0,人気_24.0
0,198601010101,1986-06-07,0.316457,1.0,ワクセイ,0.305157,0.393765,0.070510,0.527279,0,...,0,0,0,0,0,0,0,0,0,0
1,198601010101,1986-06-07,0.316457,2.0,マツタカラオー,0.305157,0.393765,0.223038,0.399433,0,...,0,0,0,0,0,0,0,0,0,0
2,198601010101,1986-06-07,0.316457,3.0,カンキョウヘルス,0.305157,0.393765,0.547479,0.501247,0,...,0,0,0,0,0,0,0,0,0,0
3,198601010101,1986-06-07,0.316457,4.0,スズタカエース,0.414072,0.483403,0.070510,0.488061,0,...,0,0,0,0,0,0,0,0,0,0
4,198601010101,1986-06-07,0.316457,5.0,クリヤーパーマン,0.305157,0.393765,0.206087,0.406438,0,...,0,0,0,0,0,0,0,0,0,0


レースの日付で新しい順にソートする。

In [15]:
df_race_result['レース日付'] = pd.to_datetime(df_race_result['レース日付'])
sorted_df = df_race_result.sort_values(by='レース日付', ascending=False)
df_race_result = sorted_df
df_race_result.head()

,レースID,レース日付,距離(m),着順,馬名,馬齢,斤量,単勝,馬体重,競馬場名_中京,...,人気_15.0,人気_16.0,人気_17.0,人気_18.0,人気_19.0,人気_20.0,人気_21.0,人気_22.0,人気_23.0,人気_24.0
1600666,202102010902,2021-07-31,0.414163,7.0,バンブトンローズ,0.171856,0.347738,0.481574,0.559201,0,...,0,0,0,0,0,0,0,0,0,0
1600776,202102010911,2021-07-31,0.541041,4.0,ヒシヴィクトリー,0.506159,0.393765,0.367208,0.608916,0,...,0,0,0,0,0,0,0,0,0,0
1600769,202102010910,2021-07-31,0.142285,12.0,エムオーシャトル,0.171856,0.253119,0.364726,0.474759,0,...,0,0,0,0,0,0,0,0,0,0
1600770,202102010910,2021-07-31,0.142285,13.0,マリノディアナ,0.506159,0.393765,0.683876,0.461340,0,...,1,0,0,0,0,0,0,0,0,0
1600771,202102010910,2021-07-31,0.142285,14.0,ラキ,0.305157,0.393765,0.564725,0.507797,0,...,0,0,0,0,0,0,0,0,0,0


処理後のデータをCSVで出力する。

In [ ]:
df_race_result.to_csv('data/prepared.csv', index=False, encoding='utf-8-sig')